In [1]:
import pandas as pd

df = pd.read_csv("Training_set.csv")

In [2]:
import os,shutil
import numpy as np
from_dir = os.getcwd()+"/"+"train"
for i in np.unique(df["label"]).tolist():
    if not os.path.isdir(from_dir+"/" +i):
        os.mkdir(from_dir+"/" +i)
        
for i in os.listdir(from_dir):
    if ".jpg" in i:
        shutil.copy(from_dir+"/" +i,from_dir +"/" +df[df["filename"]==i]["label"].tolist()[0]+"/"+i)
        os.remove(from_dir+"/" +i)

In [18]:
# import cv2
# for i in os.listdir(os.getcwd()+"/"+"train"):
#     if os.path.isdir(os.getcwd()+"/"+"train"):
#         for j in os.listdir(os.getcwd()+"/"+"train"+"/"+i):
#             img = cv2.imread(os.getcwd()+"/"+"train"+"/"+i+"/"+j)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,        # 隨機旋轉的角度範圍
    width_shift_range=0.3,    # 水平方向隨機平移的範圍
    height_shift_range=0.3,   # 垂直方向隨機平移的範圍
    shear_range=0.3,          # 剪切強度 (shear transformation)
    zoom_range=0.3,           # 隨機縮放的範圍
    horizontal_flip=True,     # 水平翻轉
    brightness_range=[0.8, 1.2]  # 亮度調整範圍
)


In [19]:
import tensorflow as tf
import os
from_dir = os.getcwd()+"/"+"train"
source_path = from_dir
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#         source_path,
#         label_mode ='categorical',
#         color_mode = 'rgb',
#         batch_size = 32,
#         image_size = (224, 224),
#         shuffle=True,
#         seed = 13,
#         validation_split = 0.2,
#         subset='training'
#     )
# valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
#         source_path,
#         label_mode ='categorical',
#         color_mode = 'rgb',
#         batch_size = 32,
#         image_size = (224, 224),
#         shuffle=True,
#         seed = 13,
#         validation_split = 0.2,
#         subset='validation'
#     )
# # 
image_size = (224, 224)
batch_size = 128
train_datagen = ImageDataGenerator(rescale = 1./255,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            rotation_range = 5,
                            horizontal_flip = True,
                            # vertical_flip = True,
                            validation_split = 0.2)

train_ds = train_datagen.flow_from_directory(source_path,
                                      target_size = image_size,
                                      batch_size = batch_size,
                                      class_mode = 'sparse',
                                      subset = 'training',
                                      color_mode="rgb",
                                      shuffle=True)

val_ds = train_datagen.flow_from_directory(source_path,
                                      target_size = image_size,
                                      batch_size = batch_size,
                                      class_mode = 'sparse',
                                      subset = 'validation',
                                      color_mode="rgb")

Found 5225 images belonging to 75 classes.
Found 1274 images belonging to 75 classes.


In [20]:
from keras import applications
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from keras.utils import np_utils
vgg_base = applications.VGG19(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
vgg_base.trainable = False
def create_model():
    
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(224, 224, 3)),
        # tf.keras.layers.experimental.preprocessing.Rescaling(1/255, input_shape=(224, 224, 3)),
        vgg_base,
        # tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        # tf.keras.layers.MaxPooling2D(2,2),
        # tf.keras.layers.MaxPooling2D(2,2),
        # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        # tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
        # tf.keras.layers.MaxPooling2D(2,2),
        # tf.keras.layers.MaxPooling2D(2,2),
        # tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        # tf.keras.layers.MaxPooling2D(2,2),
        # tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        # tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.GlobalAveragePooling2D(),
        # tf.keras.layers.Dense(32, activation='relu'),
        # tf.keras.layers.Dropout(0.3),
        # tf.keras.layers.Dense(512, activation='relu'),
        # tf.keras.layers.Dropout(0.3),
        # tf.keras.layers.Dropout(0.1),
        # tf.keras.layers.Dense(2048, activation='relu'),
        # tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1024, activation='elu'),
        # tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(75, activation='softmax')
    ])
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [21]:
model = create_model()
model.build(input_shape=(None, 224, 224, 3))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d_3  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_6 (Dense)             (None, 1024)              525312    
                                                                 
 dense_7 (Dense)             (None, 75)                76875     
                                                                 
Total params: 20626571 (78.68 MB)
Trainable params: 602187 (2.30 MB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


In [ ]:
import PIL
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)
history = model.fit(train_ds, validation_data=val_ds, epochs=100,shuffle=True,callbacks=callback)

Epoch 1/100
41/41 [==============================] - 158s 4s/step - loss: 3.8939 - accuracy: 0.1470 - val_loss: 3.1703 - val_accuracy: 0.3273
Epoch 2/100
41/41 [==============================] - 156s 4s/step - loss: 2.6869 - accuracy: 0.4285 - val_loss: 2.1846 - val_accuracy: 0.5228
Epoch 3/100
41/41 [==============================] - 155s 4s/step - loss: 1.9598 - accuracy: 0.5661 - val_loss: 1.6762 - val_accuracy: 0.6248
Epoch 4/100
41/41 [==============================] - 155s 4s/step - loss: 1.5601 - accuracy: 0.6387 - val_loss: 1.4115 - val_accuracy: 0.6625
Epoch 5/100
41/41 [==============================] - 155s 4s/step - loss: 1.3233 - accuracy: 0.6827 - val_loss: 1.2581 - val_accuracy: 0.6954
Epoch 6/100
41/41 [==============================] - 155s 4s/step - loss: 1.1753 - accuracy: 0.7043 - val_loss: 1.1516 - val_accuracy: 0.6970
Epoch 7/100
41/41 [==============================] - 155s 4s/step - loss: 1.0759 - accuracy: 0.7275 - val_loss: 1.0607 - val_accuracy: 0.7221
Epoch 

In [ ]:
import datetime
model.save(os.getcwd()+"/"+"models/model_VGG19 "+datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
     